# 勝式とオッズのデータをクロール

In [1]:
from datetime import datetime
from datetime import timedelta
from http.client import RemoteDisconnected
from bs4 import BeautifulSoup
import urllib.request
import time
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm

## 1. モジュールをロード

In [2]:
def make_url(crawl_key, rno, jcd, hd):
    """
    :param crawl_key: 何をcrawleするか。選択肢は、"odds3t"（オッズ）, "racelist"(出走表）,
    "beforeinfo" (直前情報）もしくは"raceresult" (レース結果)
    :param rno: レース番号。8Rなど、1-12の数字 + R をstrで
    :param jcd: 会場名。"桐　生"、"びわこ"など
    :param hd: holding day (レース開催日)、2019/03/28などyyyy/mm/ddの形で入力（strで）
    :return dds_url: 公式サイト最終オッズが書かれているページのurl. これを使ってcrawlする
    """
    jcd_dict =  {"桐　生": "01", "戸　田": "02", "江戸川": "03", "平和島": "04", "多摩川": "05", "浜名湖": "06", "蒲　郡": "07", "常　滑": "08",
                "　津　": "09", "三　国": "10", "びわこ": "11", "住之江": "12", "尼　崎": "13", "鳴　門": "14", "丸　亀": "15", "児　島": "16",
                "宮　島": "17", "徳　山": "18", "下　関": "19", "若　松": "20", "芦　屋": "21", "福　岡": "22", "唐　津": "23", "大　村": "24"
                }
    rno = rno[:-1]
    hd = hd[0:4] + hd[5:7] + hd[8:10]

    odds_url = "http://boatrace.jp/owpc/pc/race/" + crawl_key + "?rno=" + rno + "&jcd=" + jcd_dict[jcd] + "&hd=" + hd

    return odds_url


def html_parser(site_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
    }

    try:
        request = urllib.request.Request(url=site_url, headers=headers)
        response = urllib.request.urlopen(request)

        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'lxml')

    # データベース作成の際、remotedisconnectedになった場合,そのレースをパス
    except RemoteDisconnected:
        print("remote disconnected error !")
        return None

    except ConnectionResetError:
        print("Connection Reset error !")
        return None

    return soup

def get_extractor(crawl_key):
    
    """
    クロール先に応じたcrawlerを用意
    
    """
    
    extractor_dict = {"racelist": scrape_racelist,
                      "beforeinfo": scrape_beforeinfo,
                      "raceresult": scrape_raceresult,
                      }
    
    return extractor_dict[crawl_key]

## 2. 実行
- クロール元：ボートレース 公式サイト（https://boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=01&hd=20210325など）
- 保存先：'./crawledData/　以下。日にちごとにファイルを作成し保存

### 2.1 レースデータのみがある日付をピックアップ

In [3]:
import glob
import os
race_file_set = set([os.path.basename(file) for file in glob.glob(os.path.join('../../data/crawledData', '*.pkl'))])
win_and_pay_file_set = set([os.path.basename(file) for file in glob.glob(os.path.join('../../data/crawledData/win_and_pay', '*.pkl'))])

diff_file_name_list = list(race_file_set - win_and_pay_file_set)
hd_list = [filename[0:4] + "/" + filename[4:6] + "/" + filename[6:8] for filename in diff_file_name_list]
print("{0}日分のデータが未収集".format(len(hd_list)))

67日分のデータが未収集


### 2.2 実行

In [4]:
jcd_list =  ["桐　生", "戸　田", "江戸川", "平和島", "多摩川", "浜名湖", "蒲　郡", "常　滑",
                "　津　", "三　国", "びわこ", "住之江", "尼　崎", "鳴　門", "丸　亀", "児　島",
                "宮　島", "徳　山", "下　関", "若　松", "芦　屋", "福　岡", "唐　津", "大　村"
            ]

for hd in hd_list:
    
    win_and_pay_dict_list = []

    print("{0} のデータをクロール中".format(hd))

    # 1日単位でデータを集めてファイルに保存する
    today_race_df_list = []

    for jcd in tqdm(jcd_list):
        for i in range(1, 13):
            rno = str(i) + "R"

            # その日レースがない場所は飛ばすためのtry-except         
            try:
                raceResult_url = make_url("raceresult", rno, jcd, hd)

                # パース
                soup = html_parser(raceResult_url)

                # 対象サイトをcrawl
                table = soup.find_all(class_="is-w495")[2].find_all("tbody")
                win_and_pay_dict = {"date": "-".join([hd[0:4], hd[5:7], hd[8:10]]),
                                    "venue": jcd,
                                    "raceNumber": rno[:-1]
                                   }

                win_and_pay_dict["3連単"] = [row.text for row in table[0].find_all(class_=re.compile("numberSet1_number"))]
                win_and_pay_dict["3連単odds"] = table[0].find(class_="is-payout1").text

                win_and_pay_dict["3連複"] = [row.text for row in table[1].find_all(class_=re.compile("numberSet1_number"))]
                win_and_pay_dict["3連複odds"] = table[1].find(class_="is-payout1").text

                win_and_pay_dict["2連単"] = [row.text for row in table[2].find_all(class_=re.compile("numberSet1_number"))]
                win_and_pay_dict["2連単odds"] = table[2].find(class_="is-payout1").text

                win_and_pay_dict["2連複"] = [row.text for row in table[3].find_all(class_=re.compile("numberSet1_number"))]
                win_and_pay_dict["2連複odds"] = table[3].find(class_="is-payout1").text

                table_wide = table[4].find_all("tr")
                for i in range(3):
                    row_wide = table_wide[i]
                    win_and_pay_dict["拡連複_{0}".format(i+1)] = [row.text for row in row_wide.find_all(class_=re.compile("numberSet1_number"))]
                    win_and_pay_dict["拡連複odds_{0}".format(i+1)] = row_wide.find(class_="is-payout1").text

                win_and_pay_dict["単勝"] = table[5].find(class_=re.compile("numberSet1_number")).text
                win_and_pay_dict["単勝odds"] = table[5].find(class_="is-payout1").text

                table_place = table[6].find_all("tr")
                for i in range(2):
                    row_place = table_place[i]
                    win_and_pay_dict["複勝_{0}".format(i+1)] = row_place.find(class_=re.compile("numberSet1_number")).text
                    win_and_pay_dict["複勝odds_{0}".format(i+1)] = row_place.find(class_="is-payout1").text
                    
                # 今回のレースのデータを本日のデータを集めたリストに格納
                win_and_pay_dict_list.append(win_and_pay_dict)

            except IndexError:
                # print(hd + " " + jcd + rno +"データなし")
                pass
            except AttributeError:
                pass

    # dictを入れたlistをdfに変換
    win_and_pay_df = pd.DataFrame.from_dict(win_and_pay_dict_list)
    win_and_pay_df.set_index(["date", "venue", "raceNumber"], inplace=True)

    # pickleファイルで保存
    win_and_pay_df.to_pickle('../../data/crawledData/win_and_pay/{0}.pkl'.format("".join(hd.split("/"))))

2020/11/18 のデータをクロール中


  0%|          | 0/24 [00:00<?, ?it/s]

2020/10/03 のデータをクロール中


  0%|          | 0/24 [00:00<?, ?it/s]

2021/02/20 のデータをクロール中


...

  0%|          | 0/24 [00:00<?, ?it/s]

2021/02/10 のデータをクロール中


  0%|          | 0/24 [00:00<?, ?it/s]

2020/11/14 のデータをクロール中


  0%|          | 0/24 [00:00<?, ?it/s]

In [5]:
win_and_pay_df = pd.DataFrame.from_dict(win_and_pay_dict_list)
win_and_pay_df.set_index(["date", "venue", "raceNumber"], inplace=True)
win_and_pay_df

3連単  3連単odds        3連複  3連複odds     2連単  \
date       venue raceNumber                                                   
2020-11-14 戸　田   1           [1, 2, 3]   ¥1,200  [1, 2, 3]     ¥480  [1, 2]   
                 2           [1, 5, 2]   ¥9,220  [1, 2, 5]   ¥1,740  [1, 5]   
                 3           [3, 1, 5]  ¥43,040  [1, 3, 5]  ¥10,470  [3, 1]   
                 4           [4, 3, 5]  ¥10,560  [3, 4, 5]   ¥1,140  [4, 3]   
                 5           [3, 1, 4]   ¥5,580  [1, 3, 4]   ¥1,510  [3, 1]   
...                                ...      ...        ...      ...     ...   
           大　村   8           [6, 1, 2]   ¥4,750  [1, 2, 6]     ¥440  [6, 1]   
                 9           [1, 3, 4]   ¥1,000  [1, 3, 4]     ¥640  [1, 3]   
                 10          [1, 2, 5]   ¥1,280  [1, 2, 5]     ¥810  [1, 2]   
                 11          [1, 3, 2]   ¥1,080  [1, 2, 3]     ¥390  [1, 3]   
                 12          [1, 3, 4]     ¥730  [1, 3, 4]     ¥380  [1, 3]   

                            2連単odds     2連複 2連複odds   拡連複_1 拡連複odds_1   拡連複_2  \
date       venue raceNumber                                                     
2020-11-14 戸　田   1             ¥450  [1, 2]    ¥410  [1, 2]      ¥260  [1, 3]   
                 2           ¥2,620  [1, 5]  ¥1,300  [1, 5]      ¥210  [1, 2]   
                 3           ¥2,410  [1, 3]    ¥820  [1, 3]      ¥200  [3, 5]   
                 4           ¥4,970  [3, 4]  ¥1,480  [3, 4]      ¥430  [4, 5]   
                 5           ¥1,300  [1, 3]    ¥760  [1, 3]      ¥340  [3, 4]   
...                             ...     ...     ...     ...       ...     ...   
           大　村   8             ¥920  [1, 6]    ¥250  [1, 6]      ¥130  [2, 6]   
                 9             ¥360  [1, 3]    ¥350  [1, 3]      ¥240  [1, 4]   
                 10            ¥190  [1, 2]    ¥150  [1, 2]      ¥150  [1, 5]   
                 11            ¥330  [1, 3]    ¥210  [1, 3]      ¥180  [1, 2]   
                 12            ¥320  [1, 3]    ¥280  [1, 3]      ¥120  [1, 4]   

                            拡連複odds_2   拡連複_3 拡連複odds_3 単勝  単勝odds 複勝_1  \
date       venue raceNumber                                               
2020-11-14 戸　田   1               ¥210  [2, 3]      ¥570  1    ¥250    1   
                 2               ¥220  [2, 5]      ¥210  1    ¥240    1   
                 3             ¥1,380  [1, 5]      ¥270  3    ¥550    3   
                 4               ¥460  [3, 5]      ¥190  4  ¥1,020    4   
                 5               ¥360  [1, 4]      ¥680  3    ¥260    3   
...                               ...     ...       ... ..     ...  ...   
           大　村   8               ¥390  [1, 2]      ¥150  6    ¥510    6   
                 9               ¥260  [3, 4]      ¥510  1    ¥120    1   
                 10              ¥260  [2, 5]      ¥630  1    ¥110    1   
                 11              ¥240  [2, 3]      ¥250  1    ¥100    1   
                 12              ¥140  [3, 4]      ¥290  1    ¥130    1   

                            複勝odds_1 複勝_2 複勝odds_2  
date       venue raceNumber                         
2020-11-14 戸　田   1              ¥120    2     ¥150  
                 2              ¥260    5     ¥290  
                 3              ¥160    1     ¥170  
                 4              ¥240    3     ¥150  
                 5              ¥200    1     ¥170  
...                              ...  ...      ...  
           大　村   8              ¥140    1     ¥100  
                 9              ¥100    3     ¥160  
                 10             ¥100    2     ¥100  
                 11             ¥100    3     ¥280  
                 12             ¥100    3     ¥140  

[156 rows x 20 columns]